In [ ]:
import pandas as pd
import toml
import re
import pygsheets
from tqdm.auto import tqdm
import Bio.Restriction as Restriction
import benchlingapi

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import paulssonlab.api as api
import paulssonlab.api.benchling as bapi
from paulssonlab.api.util import base_url
import paulssonlab.cloning.workflow as workflow
import paulssonlab.cloning.util as cloning_util
import paulssonlab.cloning.io as cloning_io
import paulssonlab.cloning.sequence as sequence
import paulssonlab.cloning.registry as registry
import paulssonlab.cloning.enzyme as enzyme
import paulssonlab.cloning.commands.semantics as cmd_semantics
import paulssonlab.cloning.commands.parser as cmd_parser
import paulssonlab.api.benchling as benchling

# Setup

In [ ]:
config = toml.load("config.toml")

In [ ]:
gc = pygsheets.authorize(service_account_file="credentials.json")

In [ ]:
bench_session = benchlingapi.Session(config["benchling"]["api_key"])
benchling_folder = bapi.get_project_root(bench_session, config["benchling"]["project"])

In [ ]:
reg = registry.Registry(gc, config["registry"]["folder"], benchling_folder)

# Rename .gb to .gbk

In [ ]:
files = api.google.list_drive(reg.drive_service, root=reg.gdrive_ids[("pLIB", "maps")])

In [ ]:
service = build("drive", "v3", credentials=reg.sheets_client.oauth)
batch = service.new_batch_http_request()
for name, file in files.items():
    if name.endswith(".gb"):
        batch.add(
            reg.drive_service.files().update(
                fileId=file["id"], body={"name": f"{name}k"}
            )
        )
batch.execute()

# Plasmid map for Daniel

In [ ]:
# test pcr with other primer order

In [ ]:
c = reg[("LIB", "parts")]

In [ ]:
cmd_semantics.eval_exprs_by_priority("oLT72=oLT73", reg.get)

In [ ]:
cmd_semantics.eval_expr("pLIB122", reg.get)

In [ ]:
cmd_semantics.eval_expr("JUMP_upstream_T97", reg.get)

In [ ]:
plib122 = cmd_semantics.eval_expr("pLIB122", reg.get)["_seq"]
olib179 = cmd_semantics.eval_expr("oLIB179", reg.get)["_seq"]
olib180 = cmd_semantics.eval_expr("oLIB180", reg.get)["_seq"]
olib181 = cmd_semantics.eval_expr("oLIB181", reg.get)["_seq"]
olib182 = cmd_semantics.eval_expr("oLIB182", reg.get)["_seq"]
a = cmd_semantics.eval_expr("pLIB122<oLIB182,oLIB179>", reg.get)["_seq"]
b = cmd_semantics.eval_expr("JUMP_upstream_T97", reg.get)["_seq"]
c = cmd_semantics.eval_expr("pLIB122<oLIB180,oLIB181>", reg.get)["_seq"]
d = cmd_semantics.eval_expr("JUMP_downstream_T540", reg.get)["_seq"]

In [ ]:
olib180

In [ ]:
olib181

In [ ]:
a[:20]

In [ ]:
b[:20]

In [ ]:
len(a)

In [ ]:
len(c)

In [ ]:
len(plib122)

In [ ]:
cd_nocirc = sequence.assemble(
    [c.annotate("c"), d.annotate("d")], method="gibson", circularize=True
)

In [ ]:
cd_nocirc

In [ ]:
cd_nocirc._assemble(None, method="gibson")

In [ ]:
ab = sequence.assemble(
    [a.annotate("a"), b.annotate("b", type="foo")], method="gibson", circularize=False
)

In [ ]:
from Bio.Seq import Seq

In [ ]:
x = sequence.DsSeqRecord(Seq("gagcctttcgttttatttgatg")).annotate("a")
y = b.annotate("b", type="foo").slice(10, None, annotation_start=0)
(x + y).features

In [ ]:
y.features

In [ ]:
ab = sequence.assemble(
    [
        sequence.DsSeqRecord(Seq("gagcctttcgttttatttgatg")).annotate("a"),
        b.annotate("b", type="foo"),
    ],
    method="gibson",
    circularize=False,
)

In [ ]:
ab.features

In [ ]:
bc = sequence.assemble([b.annotate("b"), c.annotate("c")], method="gibson")

In [ ]:
with open("/Users/jacob/Downloads/_ab2.gb", "w") as f:
    f.write(ab.format("genbank"))

In [ ]:
sequence.assemble([b, c], method="gibson")

In [ ]:
sequence.assemble([c, d], method="gibson")

In [ ]:
sequence.assemble([d, a], method="gibson")

In [ ]:
s = cmd_semantics.eval_command(
    "@Gib(pLIB122<oLIB182,oLIB179>, JUMP_upstream_T97, pLIB122<oLIB180,oLIB181>, JUMP_downstream_T540)",
    reg.get,
)

In [ ]:
s = cmd_semantics.eval_command(
    "@Gib(pLIB122<oLIB182,oLIB179>, JUMP_upstream_2r2, pLIB122<oLIB180,oLIB181>, JUMP_downstream_2r2)",
    reg.get,
)

In [ ]:
with open("/Users/jacob/Downloads/insulated_hudson.gb", "w") as f:
    f.write(s["_seq"].format("genbank"))

In [ ]:
pcrs = [
    "pLIB122<oLIB182,oLIB179>",
    "pLIB122<oLIB180,oLIB181>",
    "pLIB122<oLIB180,oLIB179>",
    "pLIB122<oLIB182,oLIB181>",
]
seqs = [cmd_semantics.eval_expr(c, reg.get) for c in pcrs]
[len(s["_seq"]) for s in seqs]

In [ ]:
reg[("oLT", "oligos")].clear_cache()

In [ ]:
s = cmd_semantics.eval_command("@Gib(oLT78, oLT79)", reg.get)

In [ ]:
s2 = cmd_semantics.eval_expr("JUMP_upstream_2r2", reg.get)

In [ ]:
s["_seq"]

In [ ]:
s2["_seq"]

In [ ]:
s2["_seq"] == s["_seq"]

# Test case

In [ ]:
c = reg[("pTEST", "maps")]

In [ ]:
c.set_from_file(
    "pTEST1", "/Users/jacob/Downloads/addgene-plasmid-59954-sequence-95195.dna"
)
c.set_from_file(
    "pTEST2", "/Users/jacob/Downloads/addgene-plasmid-59954-sequence-95195.gbk"
)
c.set_from_file(
    "pTEST3.bad", "/Users/jacob/Downloads/addgene-plasmid-59954-sequence-95195.gbk"
)
c.set_from_file(
    ("foo_dir", "pTEST3.bad"),
    "/Users/jacob/Downloads/addgene-plasmid-59954-sequence-95195.gbk",
)

In [ ]:
list(c.local.keys())

In [ ]:
seq1 = cloning_io.read_file(
    "/Users/jacob/Downloads/addgene-plasmid-59954-sequence-95195.dna"
)

In [ ]:
c["dir1"] = {"a": seq1, "b": seq1}
c[("b_dir", "dir2")] = {"a": seq1, "b": seq1}
c.content[("b_dir", "dir3")] = {"a.gbk": seq1, "b.gbk": seq1}

In [ ]:
c.raw[("c_dir", "c.txt")] = {"mimeType": "plain/text", "content": "blah"}
c.bytes[("c_dir", "c.txt")]

In [ ]:
c.raw["test3.gb"] = None
c["test4"] = None

In [ ]:
c["test4"] = "foobar"

In [ ]:
x = ("b_dir", "dir2", "a.gbk")
folders = set()
folders.update(x[:i] for i in range(1, len(x)))

In [ ]:
folders

In [ ]:
set.update?

In [ ]:
set(sorted(set([("b", 1), ("a", 0)])))

In [ ]:
list(c.local.keys())

# Test

In [ ]:
del reg.clients[("pTEST", "maps")]

In [ ]:
# reg.duplicate_collection("LIB", "TEST")

In [ ]:
reg.refresh()

In [ ]:
c = reg[("pTEST", "maps")]

In [ ]:
c.clear_cache()

In [ ]:
c._download()

In [ ]:
c._remote

In [ ]:
c[()]

In [ ]:
c.client.files().update(
    fileId="1QTyMeRMLF8F3Hf21Dmc3PDY_3PPhA2U1", body={"trashed": True}
).execute()

In [ ]:
api.google.delete_drive(c.client, "1QTyMeRMLF8F3Hf21Dmc3PDY_3PPhA2U1")

In [ ]:
c._remote_folders

In [ ]:
sorted(c._remote_folders)

In [ ]:
c[("bar", "bar1", "plib84")]

In [ ]:
c.raw[("bar", "bar1", "plib84.gb")]

In [ ]:
c.keys()

In [ ]:
c.content["plib84.gb"]

In [ ]:
c["test3"] = None

In [ ]:
c._remote

In [ ]:
c.bytes["test3.gb"] = 

# Benchling test

In [ ]:
z = reg.sync_benchling(overwrite=True, return_data=True)

In [ ]:
reg.duplicate_collection("LIB", "LT", clear=True)

In [ ]:
reg.registry

# PCR simulation test

In [ ]:
seq1 = "TTTT"
seq1a = "TTTTA"
seq1b = "TTTTC"
seq1c = "CCCCCCCCCCCTTTTC"
seq2 = "AAAATTTTAAAATTTTAAAA"
seq3 = "AGTGATTTTTTTCTCCATTCTTTGTGTGTTTTTTTTGTTTTATGAATTTTTTTAACTGATACCCGTTTTTTTGGAAGGAGACCCGTTTTTTTGGAAG"
seq4 = "TTTTTTAAAAAAAGGGGGGGGGGGGGGTTTTTTCCCCCCCCCCCCAAAAAAATTTTTTAAAAAAA"
seq4p = "GGGGGGGGGGGGGG"

In [ ]:
sequence.find_primer_binding_site(seq4p, seq4)

In [ ]:
sequence.slice_seq?

In [ ]:
sequence.find_primer_binding_site(seq5, p2)

In [ ]:
p1 = "TTTTTAGAAGA"
p2 = "TTTTTTAGAGGG"
seq5 = "AGAAGACCCCGGGGCCCTCT"
sequence.pcr(seq5, p1, p2)

In [ ]:
p1 = "ttttttatggacaaag"
p2 = "ttttttcgatacgggg"
seq6 = "atggacaaagactttgatttcgataatattggcaaacgcaccccgtatcg"
sequence.pcr(seq6, p1, p2)

In [ ]:
sequence.find_primer_binding_site(seq6, p2, try_reverse_complement=True)

In [ ]:
sequence.find_primer_binding_site(
    seq6, sequence.reverse_complement(p2), try_reverse_complement=False
)

In [ ]:
sequence.reverse_complement(seq5)

# Circularization test

In [ ]:
a = sequence.DsSeqRecord(
    "abcdxxxx" + "a" * 20 + "g" * 20 + "zabcdxxxx",
    upstream_overhang=-7,
    downstream_overhang=14,
)
a

In [ ]:
sequence._assemble_gibson(a, a, max_overlap=40)

In [ ]:
sequence._assemble_gibson(a, None)

In [ ]:
a.assemble(None, method="gibson")

In [ ]:
b = sequence.DsSeqRecord(
    "abcdxxxx" + "a" * 20 + "g" * 20 + "zabcd",
    upstream_overhang=-4,
    downstream_overhang=4,
)
b

In [ ]:
print(_)

In [ ]:
b.assemble(None, method="goldengate")

In [ ]:
print(_)

# Command parsing

In [ ]:
commands.expr_parser.parse("oLIT1=oLIT2<oLIB1,oLIB2>/BsaI")

In [ ]:
commands.expr_parser.parse(
    "oLIT1=oLIT2<oLIB1,oLIB2>/BsaI,oLIT1=oLIT2<oLIB1,oLIB2>/BsaI"
)

In [ ]:
commands.command_parser.parse(
    "@3(foo, bar, baz, oLIT1=oLIT2<oLIB1,oLIB2>/BsaI, oLIT1=oLIT2<oLIB1,oLIB2>/BsaI)"
)

In [ ]:
command = "@Gib(@GG(UNS1, J23101, BCD11, UNS5), pLIB47<oLIB22,oLIB24>/BsaI)"
ast = commands.command_parser.parse(command)
ast

In [ ]:
# command = (
#     "@3G(UNS1-J23101-BCD11-mVenus-L3S3P11-UNS5, UNS5-J23150-CFP-BCD16-L3S2P55-UNS10)"
# )
# command = "@Gib(@GG(UNS1, J23101, BCD11, UNS5), pLIB47~oLIB22,oLIB24/BsaI)"
# command = "@Gib(@GG(UNS1, J23101, BCD11, UNS5), @PCR(pLIB47, oLIB22, oLIB24)/BsaI)"
# command = (
#     "@Gib(@GG(UNS1, J23101, BCD11, UNS5), @RE(@PCR(pLIB47, oLIB22, oLIB24), BsaI))"
# )

In [ ]:
# GG
# command = "@GG(J23101, BCD11, mVenus, L3S3P11, p121/BsaI)"
# 3G
# command = "@3G(@GG(UNS1, J23101, BCD11, mVenus, L3S3P11, UNS5), @GG(UNS5, J23150, CFP, BCD16, L3S2P55, UNS10), JUMP_p15a_UNS1_UNS10)"
# command = """@Gib(@GG:tLT(UNS1_A, pTac, B0032m_BC, sigW, L3S1P13, UNS3_E),
#                   @GG:tLT(UNS3_A, J23107_AB, B0034m_BC, mScarlet-I, L3S3P11, UNS4_E),
#                   @GG:tLT(UNS4_A, PsigW, B0032m_BC, sfGFP, L3S2P55, UNS5_E),
#                   @GG:tLT(UNS5_A, pTet_AB, B0034m_BC, rsiW, L3S2P11, UNS6_E),
#                   @GG:tLT(UNS6_A, PsigW, BCD16, sigW, L3S2P21, UNS10_E))"""
# TODO: pLIB213-pLIB218 maps
# TODO: BACKBONE!!!
# command = """@Gib(@GG:tLT3(UNS1_A, pTac, B0032m_BC, sfCFP, L3S1P13, UNS3_E),
#                   @GG:tLT(UNS3_A, J23107_AB, B0034m_BC, mScarlet-I, L3S3P11, UNS4_E))"""
# command = "@GG:tLT(UNS1_A, pTac, B0032m_BC, sfCFP, L3S1P13, UNS3_E)"
command = """@Gib(@GG:tLT(UNS1_A, pTac, B0032m_BC, sfCFP, L3S1P13, UNS3_E),
                  @GG:tLT(UNS3_A, J23107_AB, B0034m_BC, mScarlet-I, L3S3P11, UNS4_E),
                  @GG:tLT(UNS4_A, pTac, B0032m_BC, sfGFP, L3S2P55, UNS5_E),
                  @GG:tLT(UNS5_A, pTet_AB, B0034m_BC, sfCFP, L3S2P11, UNS6_E),
                  @GG:tLT(UNS6_A, pTac, BCD16, sfCFP, L3S2P21, UNS10_E))"""

In [ ]:
commands.command_parser.parse(command)

In [ ]:
# TODO: better error message if next id fails
# ignore columns with formulae when _get_next_empty_row

In [ ]:
workflow.parse_id(" tLIB  ")

In [ ]:
workflow._get_next_empty_row(reg.get_sheet(("tLT", "tus")))

In [ ]:
workflow.get_next_collection_id(reg.get_sheet(("tLT", "tus")))